In [1]:
import solana.system_program as sp
from solana.publickey import PublicKey
from solana.keypair import Keypair
from solana.rpc.api import Client
from solana.rpc.async_api import AsyncClient
from solana.transaction import Transaction, TransactionInstruction, AccountMeta
from spl.token.async_client import AsyncToken
from spl.token.constants import TOKEN_PROGRAM_ID
from solana.rpc.types import TxOpts

from solana.transaction import Transaction, TransactionInstruction
from spl.token.instructions import (
    initialize_mint,
    InitializeMintParams,
    initialize_account,
    InitializeAccountParams,
    mint_to_checked,
    MintToCheckedParams,
    get_associated_token_address,
    create_associated_token_account,
    transfer_checked,
    TransferCheckedParams,
)


In [2]:
cli = AsyncClient('https://api.devnet.solana.com')

In [3]:
kp_mint = Keypair.generate()
kp_payer = Keypair.generate()
kp_escrow = Keypair.generate()
kp_reciever = Keypair.generate()
kp_fee_payer = Keypair.generate()


In [4]:
print(kp_mint.public_key)
print(kp_payer.public_key)
print(kp_escrow.public_key)
print(kp_reciever.public_key)
print(kp_fee_payer.public_key)

6smHRnPD9FpcqdGzSFeLfCu4cpCKrUFJcBFeJSmGcqXM
28cmV3EGDrqj5En3VJptnzpNtWAApgSKgDoQF61DkQjf
AZ7UcmP7w94e7UAxbcUjfdJ3F78cDDj3wRYUB5uBdan2
Bcm4muYBAy9uwBSRV9XY7JFmbFpokuMz7RSFkiEqEg5z
8MNEbvXgKSUmWANoxkWpVWvTZPpv8Dau2JHVApea1cP9


In [5]:
kp_mint.public_key

6smHRnPD9FpcqdGzSFeLfCu4cpCKrUFJcBFeJSmGcqXM

In [6]:
await cli.request_airdrop(kp_payer.public_key, 100000000)

{'jsonrpc': '2.0',
 'result': '4YwSdWkGfpdmpMtr48t3YUucic3pKG9iajK24usT3WEPwKP5Ez7WT8xhGBK6FprFre4ae9aPyShiaeeKDWSegB71',
 'id': 1}

In [9]:
await cli.request_airdrop(kp_mint.public_key, 100000000)

{'jsonrpc': '2.0',
 'result': '3VjB127fLuZrqmR6RFd5Au6xopLurxWPoQu67LdJJ5jEmTffMUSWHRaF9HAsEbCNsygKRkQAfBXeNQgbxMQB7fYx',
 'id': 4}

In [10]:
await cli.request_airdrop(kp_fee_payer.public_key, 100000000)

{'jsonrpc': '2.0',
 'result': '3MHJ9W68pyZvMfCSfAuNc8UyQSvGMQdeEzTj3KryT6RGrgCbVcUrkKKxhBzwxz6CQnfsydMYPYBZBUC5ka6rdUHF',
 'id': 5}

In [11]:
await cli.get_balance(kp_mint.public_key)

{'jsonrpc': '2.0',
 'result': {'context': {'apiVersion': '1.10.26', 'slot': 145913763},
  'value': 0},
 'id': 6}

In [13]:
token = await AsyncToken.create_mint(cli, kp_mint, kp_mint.public_key, 6, TOKEN_PROGRAM_ID, kp_mint.public_key)
token

In [14]:
await cli.get_balance(kp_mint.public_key)

{'jsonrpc': '2.0',
 'result': {'context': {'apiVersion': '1.10.26', 'slot': 145914002},
  'value': 98528400},
 'id': 34}

In [15]:
token.pubkey

GYdgi65nSM7i1yjqhjpSzZW4eAmf3NSff3DH3uJGDWxZ

In [16]:
await token.get_mint_info()

MintInfo(mint_authority=6smHRnPD9FpcqdGzSFeLfCu4cpCKrUFJcBFeJSmGcqXM, supply=0, decimals=6, is_initialized=True, freeze_authority=6smHRnPD9FpcqdGzSFeLfCu4cpCKrUFJcBFeJSmGcqXM)

In [17]:
print(kp_mint.public_key)
print(token.pubkey)
type(token.pubkey)

6smHRnPD9FpcqdGzSFeLfCu4cpCKrUFJcBFeJSmGcqXM
GYdgi65nSM7i1yjqhjpSzZW4eAmf3NSff3DH3uJGDWxZ


solana.publickey.PublicKey

In [18]:
kp_mint_tokaddr = get_associated_token_address(kp_mint.public_key, token.pubkey)
kp_mint_tokaddr

j8kBeWYVdBP9QPzrVtJm7LkmZ4MiPQ7Ktbtvqtag2Ep

In [19]:
# , opts={'feePayer': kp_mint}
# opts=TxOpts(skip_confirmation=False)
# opts=TxOpts(feePayer=kp_mint)
# await token.mint_to_checked(kp_payer.public_key, kp_mint.public_key, 1000, 6)

txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
    mint_to_checked(
        MintToCheckedParams(
            TOKEN_PROGRAM_ID,
            token.pubkey,
            kp_mint_tokaddr,
            kp_mint.public_key,
            10,
            6,
        )
    )
)
signers = [kp_mint]
await cli.send_transaction(txn, *signers)



RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: invalid account data for instruction', 'data': {'accounts': None, 'err': {'InstructionError': [0, 'InvalidAccountData']}, 'logs': ['Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [1]', 'Program log: Instruction: MintToChecked', 'Program log: Error: InvalidAccountData', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 1601 of 200000 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA failed: invalid account data for instruction'], 'unitsConsumed': 0}}

In [20]:

txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_mint.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)



{'jsonrpc': '2.0',
 'result': '3fsyCK2mV9a9toENN9xZKXAZzDvTVoscYt4PS9TW3N2hosroPoyUYefeX6tjASpukrfsrsBefiL3RtvP6CFn8jhm',
 'id': 39}

In [21]:
txn = Transaction(fee_payer=kp_mint.public_key)

txn.add(
    mint_to_checked(
        MintToCheckedParams(
            TOKEN_PROGRAM_ID,
            token.pubkey,
            kp_mint_tokaddr,
            kp_mint.public_key,
            10,
            6,
        )
    )
)
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': 'F35FQDLa8Tz7KGLjGFrEUfyW9jvWyMtcmPH3S42qEA8qFGYDod4Rjtgw5pYKKw1H5c1EWJykM5xavHScsUHw8HS',
 'id': 41}

In [22]:

txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_payer.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)



{'jsonrpc': '2.0',
 'result': '5W592FsobEwm1vSvXvcgqWoUizQK2sSkL4LYcX5VZVRaFWthaAqxhKUERWMM7Bo1kmEfeibyJEGJzP3Rpg2JSRft',
 'id': 43}

In [29]:
kp_payer_tokaddr = get_associated_token_address(kp_payer.public_key, token.pubkey)


txn = Transaction(fee_payer=kp_mint.public_key)

txn.add(
    mint_to_checked(
        MintToCheckedParams(
            TOKEN_PROGRAM_ID,
            token.pubkey,
            kp_payer_tokaddr,
            kp_mint.public_key,
            10,
            6,
        )
    )
)
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '5xmX2Yevydk8UM335zbP4DS118FXexLzeF8aZguVHSTHMQmkFribWQUZD4HRFaYL6Jm1532EZZQxHiQ2EECPnCie',
 'id': 54}

In [33]:
# here we can see the balance
print(token.pubkey)
print(kp_payer.public_key)
await token.get_accounts(kp_payer.public_key)


GYdgi65nSM7i1yjqhjpSzZW4eAmf3NSff3DH3uJGDWxZ
28cmV3EGDrqj5En3VJptnzpNtWAApgSKgDoQF61DkQjf


{'jsonrpc': '2.0',
 'result': {'context': {'apiVersion': '1.10.26', 'slot': 145914720},
  'value': [{'account': {'data': {'parsed': {'info': {'isNative': False,
        'mint': 'GYdgi65nSM7i1yjqhjpSzZW4eAmf3NSff3DH3uJGDWxZ',
        'owner': '28cmV3EGDrqj5En3VJptnzpNtWAApgSKgDoQF61DkQjf',
        'state': 'initialized',
        'tokenAmount': {'amount': '10',
         'decimals': 6,
         'uiAmount': 1e-05,
         'uiAmountString': '0.00001'}},
       'type': 'account'},
      'program': 'spl-token',
      'space': 165},
     'executable': False,
     'lamports': 2039280,
     'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA',
     'rentEpoch': 337},
    'pubkey': '5nmSWLqe2BZQTLKHEnWqNL6GKBjpcR5GZy6hLk1tCrVN'}]},
 'id': 58}

In [25]:
txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_reciever.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '5Mr6oVPskfd1NEyKbo2vrSS7TnfY6ANEvwueUZY6oTm5LbUxPamE1ERbeVUH1eRaV3fu25ovUzvZC4m9BJJadDfi',
 'id': 48}

In [26]:
txn = Transaction(fee_payer=kp_mint.public_key)
txn.add(
        create_associated_token_account(kp_mint.public_key, kp_escrow.public_key, token.pubkey)
    )
signers = [kp_mint]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '2z1p9uUJDtv29AyLJTmaFH4g1Ur7vUFXB1nTd8WGP5TKDZMbuU9kGr2qXrHrHNVEKgw6fp2kqofRSHXJM3nLdxkW',
 'id': 50}

In [27]:
kp_payer.public_key

28cmV3EGDrqj5En3VJptnzpNtWAApgSKgDoQF61DkQjf

In [28]:

txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=12,
            decimals=6
        )
    )
)
signers = [kp_payer]
await cli.send_transaction(txn, *signers)


RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 1}]}, 'logs': ['Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [1]', 'Program log: Instruction: TransferChecked', 'Program log: Error: insufficient funds', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 2827 of 200000 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA failed: custom program error: 0x1'], 'unitsConsumed': 0}}

In [126]:

txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=12,
            decimals=6
        )
    )
)
signers = [kp_payer]
await cli.send_transaction(txn, *signers)

# failed — too big amount

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 0: custom program error: 0x1', 'data': {'accounts': None, 'err': {'InstructionError': [0, {'Custom': 1}]}, 'logs': ['Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [1]', 'Program log: Instruction: TransferChecked', 'Program log: Error: insufficient funds', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 2827 of 1400000 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA failed: custom program error: 0x1'], 'unitsConsumed': 0}}

In [128]:

txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_reciever.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)


signers = [kp_payer]
await cli.send_transaction(txn, *signers)



{'jsonrpc': '2.0',
 'result': '2UCcqK3YLgFDcSQLXPJxzyHNwtcNcN7CU1VLZpAbMAb4KNPL84EwcHjdkCGDLBxeTjx6SFmp8nspHCgYosd5f33W',
 'id': 165}

In [137]:
# fee payer and owner are different

txn = Transaction(fee_payer=kp_fee_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)
signers = [kp_fee_payer, kp_payer]
await cli.send_transaction(txn, *signers)



{'jsonrpc': '2.0',
 'result': '2MurL5W6UWovyN99feGoyLobBWDCkJwqcyjKd2zXsMdniTnFtMbEGKcSVUpvi6ChRf2AjHtUyviQKPr1gcXeYGDB',
 'id': 178}

In [134]:

txn = Transaction(fee_payer=kp_fee_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_reciever.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)


signers = [kp_fee_payer, kp_payer]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': '5pxMLsXicC7scvE1b7cJcvwU3okRdW6vxs5s1LzTnmutdBavhsCaGbKSxzd8DCUm9n8aTpHNp2uxvd12HN5QUzME',
 'id': 174}

In [142]:
# from me to me


txn = Transaction(fee_payer=kp_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_payer.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)
signers = [kp_payer]
await cli.send_transaction(txn, *signers)


{'jsonrpc': '2.0',
 'result': 'FwCYYmvN85aNQCuYDQTrPqZmfb5gND3AeC1ajkeJusWNaqzNB7btzRPnwzfuNVrVaLsutKFSQFgMeJjano5iWu5',
 'id': 188}

In [143]:
txn = Transaction(fee_payer=kp_fee_payer.public_key)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_escrow.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)

txn.add(
    transfer_checked(
        TransferCheckedParams(
            program_id=TOKEN_PROGRAM_ID,
            source=get_associated_token_address(kp_payer.public_key, token.pubkey),
            mint=token.pubkey,
            dest=get_associated_token_address(kp_reciever.public_key, token.pubkey),
            owner=kp_payer.public_key,
            amount=1,
            decimals=6
        )
    )
)

txn